# 🎯 2D Video → 3D Gaussian Splatting Pipeline (Fixed & Enhanced)

This notebook converts a 2D video into a 3D Gaussian Splatting model (.ply) and extracts specific objects using bounding boxes.

**Pipeline Overview:**
1. ✅ Setup Environment
2. ✅ Upload & Prepare Video
3. ✅ Extract Frames
4. ✅ Run COLMAP (Camera Pose Estimation)
5. ✅ Train Gaussian Splatting Model
6. ✅ **Explore Scene & Find Perfect Bounding Box**
7. ✅ **Export Cropped Object as .ply**
8. ✅ Download Results


---
## Step 1: Check GPU & Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y colmap ffmpeg imagemagick libgl1-mesa-glx libglib2.0-0 2>/dev/null | tail -5
print("✅ System dependencies installed")

In [ ]:
import os
os.chdir('/content')

# Clone Gaussian Splatting if not already cloned
if not os.path.exists('/content/gaussian-splatting'):
    !git clone https://github.com/graphdeco-inria/gaussian-splatting --recursive -q
    print("✅ Gaussian Splatting repo cloned")
else:
    print("✅ Gaussian Splatting repo already exists")

In [ ]:
os.chdir('/content/gaussian-splatting')

# Install Python dependencies
!pip install -q plyfile==0.8.1 tqdm

# Install submodules (these require CUDA)
!pip install -q submodules/diff-gaussian-rasterization
!pip install -q submodules/simple-knn

print("✅ Python dependencies installed")

---
## Step 2: Upload & Prepare Your Video

In [ ]:
from google.colab import files
import os

print("📁 Upload your video file (MP4, MOV, AVI supported):")
uploaded = files.upload()

video_filename = list(uploaded.keys())[0]
# Corrected video_path to reflect the actual upload location
video_path = f"/content/gaussian-splatting/{video_filename}"

''' # Verify
size_mb = os.path.getsize(video_path) / (1024*1024)
print(f"✅ Uploaded: {video_filename} ({size_mb:.1f} MB)") '''

In [ ]:
# ------ OR: use a direct URL instead of uploading ------
# Uncomment and edit the lines below if you want to download from a URL

# video_url = "https://your-video-url-here.mp4"
# video_filename = "input_video.mp4"
# video_path = f"/content/{video_filename}"
# !wget -q -O {video_path} "{video_url}"
# print(f"✅ Downloaded: {video_filename}")

---
## Step 3: Extract Frames from Video

In [ ]:
import os
import glob

# ── CONFIG ─────────────────────────────────────────────────────────────
project_name = "my_scene"      # Change this to name your project
fps          = 5               # Frames per second to extract
                               # Use 2-5 for long videos, 10 for short/detailed ones
max_dimension = 1920           # Resize longest side to this (keeps aspect ratio)
# ───────────────────────────────────────────────────────────────────────

base_dir  = f"/content/data/{project_name}"
input_dir = f"{base_dir}/input"
os.makedirs(input_dir, exist_ok=True)

# Extract frames — scale preserving aspect ratio, high quality JPEG
!ffmpeg -y -i "{video_path}" \
    -vf "fps={fps},scale='if(gt(iw,ih),{max_dimension},-2)':'if(gt(iw,ih),-2,{max_dimension})'" \
    -qscale:v 2 "{input_dir}/%04d.jpg" 2>&1 | tail -5

frames = sorted(glob.glob(f"{input_dir}/*.jpg"))
print(f"\n✅ Extracted {len(frames)} frames into {input_dir}")
print("   Tip: Aim for 100–400 frames. Too few = poor reconstruction, too many = slow.")

---
## Step 4: Run COLMAP (Structure-from-Motion)
COLMAP estimates where the camera was for each frame — essential for 3D reconstruction.

In [ ]:
import os
os.chdir('/content/gaussian-splatting')

# Set offscreen rendering for headless Colab environment
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

print("🔄 Running COLMAP... (this may take 5–20 minutes depending on frame count)")

!python convert.py -s /content/data/{project_name} --no_gpu

print("\n✅ COLMAP complete!")

# Verify COLMAP outputs
sparse_dir = f"/content/data/{project_name}/sparse/0"
if os.path.exists(sparse_dir):
    files_found = os.listdir(sparse_dir)
    print(f"   COLMAP sparse model: {files_found}")
else:
    print("⚠️  WARNING: Sparse directory not found. COLMAP may have failed.")
    print("   Try: reducing fps, ensuring the video has enough texture/detail,")
    print("   or using a shorter/simpler video.")

---
## Step 5: Train Gaussian Splatting Model
> ⏱ Estimated time: **~10 min** for 7000 iterations, **~30 min** for 30000 iterations

In [ ]:
import os
os.chdir('/content/gaussian-splatting')

# ── CONFIG ─────────────────────────────────────────────────────────────
iterations       = 30000   # 7000 = fast/preview | 30000 = high quality
output_model_dir = f"/content/output/{project_name}"
# ───────────────────────────────────────────────────────────────────────

os.makedirs(output_model_dir, exist_ok=True)

print(f"🔄 Training for {iterations} iterations...")

!python train.py \
    -s /content/data/{project_name} \
    -m {output_model_dir} \
    --iterations {iterations} \
    --test_iterations {iterations} \
    --save_iterations {iterations}

print(f"\n✅ Training complete! Model saved to: {output_model_dir}")

In [ ]:
# Verify point cloud output
point_cloud_path = f"{output_model_dir}/point_cloud/iteration_{iterations}/point_cloud.ply"

if os.path.exists(point_cloud_path):
    size_mb = os.path.getsize(point_cloud_path) / (1024*1024)
    print(f"✅ Point cloud found: {point_cloud_path}")
    print(f"   File size: {size_mb:.2f} MB")
else:
    print("❌ Point cloud not found. Check training output for errors.")
    !find {output_model_dir} -name "*.ply" 2>/dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

---
## Step 6: 🔍 Explore Scene & Find the Perfect Bounding Box

This is the key step for cropping. We'll:
1. Load the full point cloud
2. Print scene statistics (min/max XYZ, centroid)
3. Visualize the point distribution per axis
4. Give you an interactive way to tune bounding box values

In [ ]:
import numpy as np
from plyfile import PlyData
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

# ── Load point cloud ──────────────────────────────────────────────────
def load_ply_numpy(path):
    """Load PLY file and return positions as numpy array."""
    ply_data = PlyData.read(path)
    vertex = ply_data['vertex']
    x = np.array(vertex['x'])
    y = np.array(vertex['y'])
    z = np.array(vertex['z'])
    positions = np.stack([x, y, z], axis=1)
    return positions, ply_data

positions, raw_ply = load_ply_numpy(point_cloud_path)
N = len(positions)
print(f"✅ Loaded {N:,} Gaussians from point cloud")

# ── Scene statistics ───────────────────────────────────────────────────
p_min  = positions.min(axis=0)
p_max  = positions.max(axis=0)
p_mean = positions.mean(axis=0)
p_std  = positions.std(axis=0)

# Percentile-based (robust to outliers)
p_05  = np.percentile(positions, 5,  axis=0)
p_95  = np.percentile(positions, 95, axis=0)
p_10  = np.percentile(positions, 10, axis=0)
p_90  = np.percentile(positions, 90, axis=0)
p_25  = np.percentile(positions, 25, axis=0)
p_75  = np.percentile(positions, 75, axis=0)

print("\n" + "="*55)
print("         SCENE BOUNDING BOX ANALYSIS")
print("="*55)
print(f"  Total Gaussians : {N:,}")
print(f"{'Metric':<12}  {'X':>10}  {'Y':>10}  {'Z':>10}")
print("-"*45)
print(f"{'Min':<12}  {p_min[0]:>10.4f}  {p_min[1]:>10.4f}  {p_min[2]:>10.4f}")
print(f"{'5th pct':<12}  {p_05[0]:>10.4f}  {p_05[1]:>10.4f}  {p_05[2]:>10.4f}")
print(f"{'10th pct':<12}  {p_10[0]:>10.4f}  {p_10[1]:>10.4f}  {p_10[2]:>10.4f}")
print(f"{'25th pct':<12}  {p_25[0]:>10.4f}  {p_25[1]:>10.4f}  {p_25[2]:>10.4f}")
print(f"{'Mean':<12}  {p_mean[0]:>10.4f}  {p_mean[1]:>10.4f}  {p_mean[2]:>10.4f}")
print(f"{'75th pct':<12}  {p_75[0]:>10.4f}  {p_75[1]:>10.4f}  {p_75[2]:>10.4f}")
print(f"{'90th pct':<12}  {p_90[0]:>10.4f}  {p_90[1]:>10.4f}  {p_90[2]:>10.4f}")
print(f"{'95th pct':<12}  {p_95[0]:>10.4f}  {p_95[1]:>10.4f}  {p_95[2]:>10.4f}"check Step 6 stats)
print(f"{'Max':<12}  {p_max[0]:>10.4f}  {p_max[1]:>10.4f}  {p_max[2]:>10.4f}")
print(f"{'Std Dev':<12}  {p_std[0]:>10.4f}  {p_std[1]:>10.4f}  {p_std[2]:>10.4f}")
print("="*55)

print("\n💡 SUGGESTED starting bounding boxes:")
print(f"   Tight (10th–90th pct)  → min={np.round(p_10,3).tolist()}, max={np.round(p_90,3).tolist()}")
print(f"   Medium (5th–95th pct)  → min={np.round(p_05,3).tolist()}, max={np.round(p_95,3).tolist()}")
print(f"   Centred (mean ± 1 std) → min={np.round(p_mean-p_std,3).tolist()}, max={np.round(p_mean+p_std,3).tolist()}")

In [ ]:
# ── Visualise point distribution per axis ─────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
axis_names = ['X', 'Y', 'Z']
colors = ['#e74c3c', '#2ecc71', '#3498db']

for i, (ax, name, color) in enumerate(zip(axes, axis_names, colors)):
    ax.hist(positions[:, i], bins=100, color=color, alpha=0.75, edgecolor='none')
    ax.axvline(p_10[i],  color='orange', linestyle='--', linewidth=1.5, label='10th pct')
    ax.axvline(p_90[i],  color='orange', linestyle='--', linewidth=1.5, label='90th pct')
    ax.axvline(p_mean[i], color='white', linestyle='-',  linewidth=2, label='mean')
    ax.set_title(f'{name} Axis Distribution', color='white', fontsize=13)
    ax.set_xlabel('Position', color='white')
    ax.set_ylabel('Count', color='white')
    ax.tick_params(colors='white')
    ax.set_facecolor('#1a1a2e')
    ax.legend(fontsize=8)

fig.patch.set_facecolor('#0f0f1a')
plt.suptitle('Point Cloud Distribution per Axis\n(dashed = 10th/90th percentile)',
             color='white', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('/content/axis_distribution.png', dpi=120, bbox_inches='tight',
            facecolor='#0f0f1a')
plt.show()
print("✅ Histogram saved to /content/axis_distribution.png")

In [ ]:
import numpy as np
from plyfile import PlyData
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

# ── 3D scatter preview (sampled for speed) ────────────────────────────
sample_n = min(20000, N)  # sample up to 20k points for plotting
idx = np.random.choice(N, sample_n, replace=False)
pts = positions[idx]

# Colour by Z height for easy orientation
z_norm = (pts[:, 2] - pts[:, 2].min()) / (np.ptp(pts[:, 2]) + 1e-8)

fig = plt.figure(figsize=(14, 5))
views = [(20, 45, 'Perspective'), (0, 0, 'XY (top)'), (0, 90, 'XZ (front)'), (90, 0, 'YZ (side)')]

for k, (elev, azim, title) in enumerate(views):
    ax = fig.add_subplot(1, 4, k+1, projection='3d')
    ax.scatter(pts[:,0], pts[:,1], pts[:,2],
               c=z_norm, cmap='plasma', s=0.3, alpha=0.6)
    ax.view_init(elev=elev, azim=azim)
    ax.set_title(title, color='white', fontsize=10)
    ax.set_facecolor('#1a1a2e')
    ax.tick_params(colors='gray', labelsize=6)
    ax.set_xlabel('X', color='gray', fontsize=7)
    ax.set_ylabel('Y', color='gray', fontsize=7)
    ax.set_zlabel('Z', color='gray', fontsize=7)

fig.patch.set_facecolor('#0f0f1a')
plt.suptitle(f'3D Point Cloud Preview ({sample_n:,} sampled points)',
             color='white', fontsize=12)
plt.tight_layout()
plt.savefig('/content/point_cloud_3d_preview.png', dpi=120, bbox_inches='tight',
            facecolor='#0f0f1a')
plt.show()
print("✅ 3D preview saved to /content/point_cloud_3d_preview.png")

---
## Step 7: 📦 Export Cropped Object as .ply

### How to find the perfect bounding box values:

1. **Look at the histograms above** — find where your target object lives vs. background noise
2. **Use the suggested ranges** printed in Step 6 as a starting point
3. **Check the 3D views** — each view shows a different angle:
   - `XY (top)` = bird's eye
   - `XZ (front)` = front face
   - `YZ (side)` = side profile
4. **Start wide, go narrow** — begin with the 5th–95th percentile range, then tighten
5. **Check the Gaussian count** printed after each crop — a good crop retains meaningful density

**Coordinate system in Gaussian Splatting:**
- `X` = left/right
- `Y` = up/down (sometimes inverted — check 3D view)
- `Z` = depth (forward/backward)

In [ ]:
import numpy as np
from plyfile import PlyData, PlyElement
import os

# ════════════════════════════════════════════
#  ✅️  SET YOUR BOUNDING BOX HERE
#     Use the statistics & plots from Step 6 to guide you.
#     Start with the 'Tight' suggestion and adjust from there.
# ════════════════════════════════════════════

bbox_min = np.array([-3.749, -1.847, 5.221])   # ← edit these three values
bbox_max = np.array([3.346, 4.992, 10.239])   # ← edit these three values

# ════════════════════════════════════════════

output_name = "cropped_object.ply"         # Output filename
output_path = f"/content/{output_name}"

# ══ Filtering function ═════════════════════
def crop_gaussians(ply_data, bbox_min, bbox_max, output_path):
    """
    Crops a Gaussian Splatting PLY file to a bounding box.
    Preserves ALL original vertex properties (SH, opacity, scale, rotation).
    """
    vertex = ply_data['vertex']
    x = np.array(vertex['x'])
    y = np.array(vertex['y'])
    z = np.array(vertex['z'])

    mask = (
        (x >= bbox_min[0]) & (x <= bbox_max[0]) &
        (y >= bbox_min[1]) & (y <= bbox_max[1]) &
        (z >= bbox_min[2]) & (z <= bbox_max[2])
    )

    n_inside = mask.sum()
    n_total  = len(x)
    print(f"   Gaussians inside bbox  : {n_inside:,} / {n_total:,}  ({100*n_inside/n_total:.1f}%)")

    if n_inside == 0:
        print("⚠️  WARNING: No Gaussians found in this bounding box.")
        print("   → Widen your bbox_min / bbox_max values.")
        return False

    # Reconstruct filtered vertex data preserving all properties
    props = vertex.data[mask]
    filtered_element = PlyElement.describe(props, 'vertex')
    PlyData([filtered_element], text=False).write(output_path)
    return True


print(f"🔄 Cropping point cloud...")
print(f"   bbox_min = {bbox_min.tolist()}")
print(f"   bbox_max = {bbox_max.tolist()}")
print()

success = crop_gaussians(raw_ply, bbox_min, bbox_max, output_path)

if success:
    out_mb = os.path.getsize(output_path) / (1024*1024)
    orig_mb = os.path.getsize(point_cloud_path) / (1024*1024)
    print(f"\n✅ Cropped model saved: {output_path}")
    print(f"   Original size : {orig_mb:.2f} MB")
    print(f"   Cropped size  : {out_mb:.2f} MB")
    print(f"   Reduction     : {100*(orig_mb-out_mb)/orig_mb:.1f}%")

In [ ]:
# ══ Iterate: try multiple bounding boxes and compare ═════════════════
# Run this cell repeatedly with different values to compare crops

def quick_crop_stats(ply_data, bbox_min, bbox_max, label=""):
    """Quickly show how many Gaussians fall inside a bounding box."""
    vertex = ply_data['vertex']
    x = np.array(vertex['x'])
    y = np.array(vertex['y'])
    z = np.array(vertex['z'])
    mask = (
        (x >= bbox_min[0]) & (x <= bbox_max[0]) &
        (y >= bbox_min[1]) & (y <= bbox_max[1]) &
        (z >= bbox_min[2]) & (z <= bbox_max[2])
    )
    n = mask.sum()
    pct = 100 * n / len(x)
    print(f"  [{label}]  {n:>8,} Gaussians  ({pct:5.1f}%)  "
          f"bbox {bbox_min.tolist()} → {bbox_max.tolist()}")

print("Bounding box comparison:")
print("-" * 80)
quick_crop_stats(raw_ply, p_10, p_90, "tight (10-90%)")
quick_crop_stats(raw_ply, p_05, p_95, "medium (5-95%)")
quick_crop_stats(raw_ply, p_mean - p_std, p_mean + p_std, "mean±1std")
quick_crop_stats(raw_ply, p_mean - 2*p_std, p_mean + 2*p_std, "mean±2std")
quick_crop_stats(raw_ply, p_min, p_max, "full scene")
print("-" * 80)
print("\nUse these as reference, then set bbox_min/bbox_max in the cell above.")

---
## Step 8: Visualise Cropped Result

In [ ]:
# Load and visualise cropped model
if os.path.exists(output_path):
    cropped_positions, _ = load_ply_numpy(output_path)
    M = len(cropped_positions)

    sample_m = min(15000, M)
    idx2 = np.random.choice(M, sample_m, replace=False)
    cpts = cropped_positions[idx2]
    z_c = (cpts[:, 2] - cpts[:, 2].min()) / (np.ptp(cpts[:, 2]) + 1e-8)

    fig = plt.figure(figsize=(14, 4))
    views = [(20, 45, 'Perspective'), (0, 0, 'XY top'), (0, 90, 'XZ front'), (90, 0, 'YZ side')]

    for k, (elev, azim, title) in enumerate(views):
        ax = fig.add_subplot(1, 4, k+1, projection='3d')
        ax.scatter(cpts[:,0], cpts[:,1], cpts[:,2],
                   c=z_c, cmap='viridis', s=0.5, alpha=0.7)
        ax.view_init(elev=elev, azim=azim)
        ax.set_title(title, color='white', fontsize=10)
        ax.set_facecolor('#1a1a2e')
        ax.tick_params(colors='gray', labelsize=6)

    fig.patch.set_facecolor('#0f0f1a')
    plt.suptitle(f'Cropped Object Preview ({M:,} Gaussians)',
                 color='white', fontsize=12)
    plt.tight_layout()
    plt.savefig('/content/cropped_preview.png', dpi=120, bbox_inches='tight',
                facecolor='#0f0f1a')
    plt.show()
    print("✅ Cropped preview saved to /content/cropped_preview.png")
else:
    print("⚠️  Cropped file not found. Run Step 7 first.")

---
## Step 9: Download All Results

In [ ]:
print("## Summary of Pipeline Parameters for Conference Paper")
print("---------------------------------------------------")

# 1. Input Data Characteristics
print("### 1. Input Data Characteristics")
print(f"- Input Video Filename: {video_filename}")
print(f"- Extracted Frames Per Second (FPS): {fps}")
print(f"- Total Number of Extracted Frames: {len(frames)}")

# 2. COLMAP Reconstruction Metrics
print("\n### 2. COLMAP Reconstruction Metrics")
print(f"- Initial Number of 3D Points (from COLMAP): {N:,} Gaussians")
# Note: Exact number of reconstructed cameras and COLMAP time need to be extracted from COLMAP logs/output if precise values are required.

# 3. Gaussian Splatting Training Details
print("\n### 3. Gaussian Splatting Training Details")
print(f"- Number of Training Iterations: {iterations:,}")
print(f"- Output Model Directory: {output_model_dir}")
# Note: Final Loss Values (L1, PSNR) and training time need to be extracted from training logs/output.

# 4. Cropping Parameters and Results
print("\n### 4. Cropping Parameters and Results")
print(f"- Bounding Box Min (X, Y, Z): {bbox_min.tolist()}")
print(f"- Bounding Box Max (X, Y, Z): {bbox_max.tolist()}")
print(f"- Number of Cropped Gaussians: {M:,}")
if 'orig_mb' in locals() and 'out_mb' in locals():
    print(f"- Original Model Size: {orig_mb:.2f} MB")
    print(f"- Cropped Model Size: {out_mb:.2f} MB")
    print(f"- Percentage of Gaussians Retained: {100*M/N:.1f}%")
    print(f"- Storage Reduction: {100*(orig_mb-out_mb)/orig_mb:.1f}%")

# 5. Qualitative Visualizations
print("\n### 5. Qualitative Visualizations Generated")
print(f"- Axis Distribution Histogram: /content/axis_distribution.png")
print(f"- Full Scene 3D Preview: /content/point_cloud_3d_preview.png")
print(f"- Cropped Object Preview: /content/cropped_preview.png")

# 6. System Specifications
print("\n### 6. System Specifications")
if torch.cuda.is_available():
    print(f"- GPU Used: {torch.cuda.get_device_name(0)}")
else:
    print("- GPU Used: Not available or not detected")

In [ ]:
from google.colab import files
import os

def safe_download(path, label):
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / (1024*1024)
        print(f"📥 Downloading {label} ({size_mb:.2f} MB)...")
        files.download(path)
    else:
        print(f"⚠️  {label} not found at {path}")

# Download full point cloud
safe_download(point_cloud_path, "Full Point Cloud")

# Download cropped object
safe_download(output_path, "Cropped Object")

# Download visualisation images
safe_download('/content/axis_distribution.png',      "Axis Distribution Plot")
safe_download('/content/point_cloud_3d_preview.png', "Full Scene 3D Preview")
safe_download('/content/cropped_preview.png',        "Cropped Object Preview")

---
## 📌 Reference: How to Find the Perfect Bounding Box

### Method 1 — Statistics-based (easiest)
Use the table printed in **Step 6** as a guide:
- If your object is the **main subject** of the scene → use **10th–90th percentile** range
- If there is significant background → use **25th–75th percentile** to focus on the core
- If the scene has outlier noise → ignore Min/Max, trust percentiles instead

### Method 2 — Histogram-based (accurate)
Look at the axis histograms:
- Find the **dense cluster** in each axis — that's your object
- The sparse tails are usually background/noise
- Set `bbox_min[axis]` = left edge of the cluster, `bbox_max[axis]` = right edge

### Method 3 — 3D view-based (visual)
Look at the 4 3D views and estimate where the object is spatially:
- `XY top view` → set X and Y bounds
- `XZ front view` → set X and Z bounds  
- `YZ side view` → set Y and Z bounds

### Method 4 — Iterative refinement (most precise)
1. Run `quick_crop_stats()` cell with progressively tighter boxes
2. A good crop keeps **20–60%** of total Gaussians (more = more context, less = cleaner)
3. Visually inspect each crop using Step 8

### ⚠️ Common Pitfalls
| Problem | Fix |
|---|---|
| 0 Gaussians in bbox | Your values are outside the scene range — check Step 6 stats |
| Crop includes background | Tighten bbox_min/max further toward the mean |
| Object is cut off | Expand bbox in the axis where it's cut |
| Y-axis is flipped | Try negating Y values (common in some COLMAP outputs) |

### 🔭 View in a 3D Gaussian Splatting Viewer
Open your `.ply` file in:
- [SuperSplat](https://playcanvas.com/supersplat/editor) (browser-based, free)
- [Luma AI Viewer](https://lumalabs.ai) (browser-based)
- [3D Gaussian Splatting Viewer](https://github.com/antimatter15/splat) (local)

These viewers will show you the actual rendered scene and let you visually determine better bounding box coordinates.